In [68]:
#Credit for the starter code goes to Samuele Mazzanti and his NOV 2022 article
#https://towardsdatascience.com/using-causal-ml-instead-of-a-b-testing-eeb1067d7fc0

#ChatGPT logo courtesy of https://commons.wikimedia.org/wiki/File:ChatGPT_logo.svg

#Python logo courtesty of https://www.python.org/community/logos/

In [17]:
#!pip install lightgbm

In [74]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor

# Generate a sample dataset with input features and target variable
data = pd.DataFrame()
data['input_feature_1'] = np.random.normal(0, 1, 1000)
data['input_feature_2'] = np.random.normal(0, 1, 1000)
data['target_variable'] = np.random.normal(10, 2, 1000)
data['campaign_A'] = np.random.choice([0, 1], size=1000, p=[0.5, 0.5])

# Assuming you have a pandas DataFrame called 'data' with input features
X = data[['input_feature_1', 'input_feature_2', 'campaign_A']]  # Assign the input features to variable 'X'
y = data['target_variable']  # Assign the target variable to variable 'y'

In [75]:
# Initialize the number of folds and a dictionary to store the folds (Samuele's code)
n_folds = 5
folds = {fold: dict() for fold in range(n_folds)}

# For each fold
for fold, (ix_train, ix_test) in enumerate(KFold(n_splits=n_folds).split(X=X)):

    # ... store test index and trained model
    folds[fold]["ix_test"] = ix_test
    folds[fold]["model"] = LGBMRegressor().fit(
        X=X.loc[ix_train, :],
        y=y.loc[ix_train]
    )


In [76]:
# Quick check of the data
print(data.head())

   input_feature_1  input_feature_2  target_variable  campaign_A
0        -0.603144         0.501586        11.161428           0
1        -0.666917         2.089140        12.649203           1
2        -0.843950        -0.709916         8.497772           1
3         0.899722         1.696887        13.638148           0
4         0.234440        -0.656006        10.430013           0


In [77]:
# Make counterfactual datasets for campaign A (More of Samuele's code)
X_zeros = X.copy()  # Create a copy of X
X_zeros.loc[X_zeros["campaign_A"] == 1, "campaign_A"] = 0  # Set campaign_A to 0 for universe A

X_ones = X.copy()  # Create a copy of X
X_ones.loc[X_ones["campaign_A"] == 0, "campaign_A"] = 1  # Set campaign_A to 1 for universe B

pred_zeros = pd.Series(index=X.index)
pred_ones = pd.Series(index=X.index)

# For each fold, use model to make predictions on test individuals and store them
for fold in folds.keys():
    ix_test = folds[fold]["ix_test"]
    model = folds[fold]["model"]

    pred_zeros.loc[ix_test] = model.predict(X_zeros.loc[ix_test, :])
    pred_ones.loc[ix_test] = model.predict(X_ones.loc[ix_test, :])


<ipython-input-77-90c5147c8230>:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pred_zeros = pd.Series(index=X.index)
<ipython-input-77-90c5147c8230>:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pred_ones = pd.Series(index=X.index)


In [78]:
# Print a sample of the output (My )
sample_fold = 0  # Select a fold to display the output
sample_ix_test = folds[sample_fold]["ix_test"]
sample_pred_zeros = pred_zeros.loc[sample_ix_test]
sample_pred_ones = pred_ones.loc[sample_ix_test]

print("Sample Fold:", sample_fold)
print("Test Indices:", sample_ix_test[:10])  # Display the first 10 test indices
print("Predictions (Universe A):", sample_pred_zeros[:10])  # Display the first 10 predictions for universe A
print("Predictions (Universe B):", sample_pred_ones[:10])  # Display the first 10 predictions for universe B

Sample Fold: 0
Test Indices: [0 1 2 3 4 5 6 7 8 9]
Predictions (Universe A): 0     9.583919
1    11.359732
2    11.210346
3    10.399683
4    11.546509
5     8.962543
6     9.673498
7    11.767661
8    10.623508
9     8.623163
dtype: float64
Predictions (Universe B): 0     9.894047
1    11.301555
2    11.069326
3     9.844753
4    11.416505
5     9.859523
6     9.391807
7    11.463028
8    10.371302
9     9.190145
dtype: float64


In [79]:
ate = (pred_ones - pred_zeros).mean()
print(ate)

0.30441971161428855
